#Section 1 Data loading and preprocessing

In [ ]:
!pip install -U -q PyDrive
!pip install pytorchtools
!pip install efficientnet_pytorch
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
# drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1xCsWLVz6ZfO1bddSL1qS5DFAu4B6Ge7p'

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('archive.zip')

id = '1u9xj8Quq5oRMU_GLBNTsVaT2B_gNAFRm'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('models.zip')


id = '1yE-5rmrWIjG4xiO3Stsj2pAG63NUoQxk'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')


!unzip archive.zip
!unzip models.zip

In [ ]:
from efficientnet_pytorch import EfficientNet
from numpy import arange
import pandas as pd
import numpy as np
from PIL import Image
import torch
import os
import os
import pandas as pd
from torchvision.io import read_image
import cv2
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from torchvision.io import read_image
import cv2
from torch.utils.data import DataLoader, Dataset
import random
import random, os
from sklearn.metrics import roc_curve
from torchvision import transforms
import torchvision.models as models
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import sklearn.utils.class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from torch import nn
import os
import cv2
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def seed_everything(seed: int):

    
    random.seed(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(5709)

In [ ]:
# pre-prcessing data

def metadata(ready_train):
  ready_train['sex']=ready_train.sex.map({'female': 0, 'male':1})
  ready_train['sex'] = ready_train['sex'].replace(np.nan, 2)
  age_group = pd.cut(ready_train.age_approx,bins=[0,10,20,30,40,50,60,70,80,100],labels=[1,2,3,4,5,6,7,8,9])
  ready_train.insert(4, "age_group", age_group)  
  ready_train['anatom_site_general_challenge']=ready_train.anatom_site_general_challenge.map({'head/neck': 1, 'upper extremity':2, 'torso':3,'lower extremity':4, 'palms/soles':5 })
  ready_train['anatom_site_general_challenge'] = ready_train['anatom_site_general_challenge'].replace(np.nan, 0)
  ready_train["age_group"] = ready_train["age_group"].cat.add_categories(0)
  ready_train["age_group"].fillna(0, inplace =True)
  return ready_train

In [ ]:
full_train = pd.read_csv('train.csv')
full_train = metadata(full_train)

In [ ]:
# pre-prcessing data
seed_everything(5709)
train = pd.read_csv('train.csv')
unique_patient_id = train["patient_id"].unique().tolist()
len(unique_patient_id)
from collections import Counter
count= Counter(train["patient_id"])
unqualified =[]
for i in count:
  if count[i]<5:
    unqualified.append(i)
unqualified_list = train["image_name"][train["patient_id"].isin(unqualified)==True].tolist()
train = train[train["patient_id"].isin(unqualified)==False]
train = train.reset_index()
qualified = train["patient_id"].unique().tolist()

     
idx =[]
for i in qualified:
  keep = train["index"][train['patient_id'] == i].tolist()[0:5]
  for j in keep:
    idx.append(j)
clean_train = train[train["index"].isin(idx)==True]
remove_train =train[train["index"].isin(idx)==False]
remove_list = remove_train["image_name"].tolist()

Diagnosed_patient = clean_train["patient_id"][clean_train["target"]== 1].unique().tolist()
free_patient = clean_train["patient_id"][clean_train["target"]== 0].unique().tolist()
classes= np.array(clean_train["target"].tolist())
val_patient_list = random.sample(Diagnosed_patient,int(len(Diagnosed_patient)*0.2))
val_patient_list_2 = random.sample(free_patient,int(len(free_patient)*0.2))
val_patient_list.extend(val_patient_list_2)
random.shuffle (val_patient_list)
qualified_val = clean_train["patient_id"][clean_train["patient_id"].isin(val_patient_list)==True].unique().tolist()
qualified_train = clean_train["patient_id"][clean_train["patient_id"].isin(val_patient_list)==False].unique().tolist()


for_train = full_train[full_train["patient_id"].isin(qualified_val)==False]
for_train.to_csv("extra_train.csv", index = False)
for_val = full_train[full_train["patient_id"].isin(qualified_val)==True]
for_val.to_csv("validation.csv", index = False)
train_patient = for_train["patient_id"].unique().tolist()
val_patient = for_val["patient_id"].unique().tolist()
Full_patient_list = full_train["patient_id"].unique().tolist()
classes= np.array(clean_train["target"].tolist())
cancer_patient = for_train["patient_id"][for_train["target"]==1].unique().tolist()
benign_patient = for_train["patient_id"][for_train["patient_id"].isin(cancer_patient) == False].unique().tolist()

In [ ]:
# Below codes comes from Roman (2020), URL: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/159476

class Microscope:
    """
    Cutting out the edges around the center circle of the image
    Imitating a picture, taken through the microscope

    Args:
        p (float): probability of applying an augmentation
    """

    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to apply transformation to.

        Returns:
            PIL Image: Image with transformation.
        """
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8), # image placeholder
                        (img.shape[0]//2, img.shape[1]//2), # center point of circle
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15), # radius
                        (0, 0, 0), # color
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)
        
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'

In [ ]:
# Data augmentation
train_dat_tran = transforms.Compose([

                                     Microscope(p=0.5),
                                     transforms.ToPILImage(),
                                     transforms.RandomRotation(45),
                                     transforms.RandomResizedCrop(224, scale =(0.8,1.0)),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomVerticalFlip(),                                    
                                     transforms.ColorJitter(brightness=24. / 255.,saturation=0.3),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229,0.224,0.225])
])                                       
val_dat_tran = transforms.Compose([
      
                                     transforms.ToPILImage(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229,0.224,0.225])
])    

In [ ]:
# Utility functions

def onehot(lab, num):
  binary =[]
  for i in lab:
    temp = list(np.zeros(num))
    temp[int(i)] =1
    binary.append(temp)
  return binary
def stack(list):
  temp = []
  for i in list:
    for j in i:
      temp.append(j)
  temp= np.vstack(temp).ravel()

  return temp
def test(x):
  return (1/(1 + np.exp(-x)))

  
def round (x, prob):
  if x>=prob:
    x = 1
  else:
    x = 0

  return x


In [ ]:
patient_class = []

for i in Full_patient_list:
  tem = full_train["target"][full_train["target"] == 1][full_train["patient_id"] == i].tolist()
  if len(tem) ==0:
    patient_class.append(0)
  else:
    patient_class.append(1)

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5,shuffle=True, random_state=2022)
testtt = StratifiedKFold(n_splits=5,shuffle=True, random_state=2022)
testtt.get_n_splits(Full_patient_list, patient_class)
kf.get_n_splits(Full_patient_list, patient_class)
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  print("TRAIN:", type(train_index), "TEST:", len(test_index))

In [ ]:
time = 0
for a,b in kf.split(Full_patient_list, patient_class):
  time+=1
  print("this is time ",time)
  for i,j in testtt.split(Full_patient_list, patient_class):
      print(np.array_equal(a,i))
      print(np.array_equal(b,j))

In [ ]:
# Data loading

class CustomImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, target_transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):

  
      img_path = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0]+".png") 




      pat_label = torch.Tensor([self.dataframe.iloc[idx, -1]])


      images = cv2.imread(img_path) 

      if self.transform is not None:
          images = self.transform(images)

      return images, pat_label

In [ ]:
# Below function comes from Brownlee (2020) but with some changes, 
# URL: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
# finding the best threshold for benign and malignan skin lesion.
def threshold(ground_truth,final_prediction):
  fpr, tpr, thresholds = roc_curve(ground_truth, final_prediction)
  gmeans = np.sqrt(tpr * (1-fpr))
  ix = np.argmax(gmeans)
  return thresholds[ix], gmeans[ix]

In [ ]:
# Loading pretrained CNNs
CON = EfficientNet.from_pretrained('efficientnet-b4')
num_ftrs = CON._fc.in_features
CON._fc = nn.Linear(num_ftrs, 1)
CON._fc.require_grad = True

RES = models.resnet50(pretrained=True)
R_num_ftrs =RES.fc.in_features
RES.fc = nn.Linear(R_num_ftrs, 1)
RES.fc.require_grad = True

#Section 2 Tranining using image feature only

##2.1 fine tuning CNNs

In [ ]:
# time = 0
# for train_index, test_index in kf.split(Full_patient_list, patient_class):
#   time +=1
#   if time <5:
#     continue

#   CON = EfficientNet.from_pretrained('efficientnet-b4')
#   num_ftrs = CON._fc.in_features
#   CON._fc = nn.Linear(num_ftrs, 1)
#   CON._fc.require_grad = True

#   RES = models.resnet50(pretrained=True)
#   R_num_ftrs =RES.fc.in_features
#   RES.fc = nn.Linear(R_num_ftrs, 1)
#   RES.fc.require_grad = True
  
#   train_patient = []
#   val_patient = []  
  
#   print("this is folder No. ", time)
#   tep = []
#   for i in test_index:
#     tep.append(Full_patient_list[i])
#     val_patient.append(Full_patient_list[i])
#   for_val = full_train[full_train['patient_id'].isin(tep)==True]
#   for_train = full_train[full_train['patient_id'].isin(tep)==False]
#   for j in train_index:
#     train_patient.append(Full_patient_list[j])
#   ones = len(for_train[for_train['target']==1])
#   zeros = len(for_train)-ones
#   a = np.zeros(zeros)
#   b = np.ones(ones)
#   c = np.concatenate((a,b))
#   random.shuffle(c)

#   class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
#   class_weights=torch.tensor(class_weights,dtype=torch.float)
#   class_weights = class_weights[1]/class_weights[0]

#   batch_size = 30
#   train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
#   val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
#   dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
#   dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


#   # Fine-tune efficientNet CNNs
#   print('Fine-tune efficientNet CNNs')
#   print("this is folder No. ", time)
#   learning_rate = 0.001
#   epochs = 10
#   model = CON
#   model = model.to(device)

#   train_roc_auc = []

#   train_loss = []

#   val_roc_auc = []

#   val_loss = []





#   optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


#   criterion = nn.BCEWithLogitsLoss()
#   scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#           optimizer,
#           patience=1,
#           factor=0.2,
#           threshold=0.001,
#           mode="max",
#           verbose=True
#       )





#   for epoch in range(epochs):
#       train_pred = []
#       train_truth=[]
      
#       final_prediction= []
#       ground_truth = []
#       print("this is epoch ", epoch+1, "running")
      
#       epoch_loss=0.0    
#       epoch_acc =0.0
      
#       #train the model
#       model.train()
      
#       batch_iteration = 0
#       for i, data in enumerate(dataloadingfortrain, 0):

#         image_in, label_in = data
#         optimizer.zero_grad()
  
        
#         label_in= label_in.to(device)

#         image_in= image_in.to(device)
      
        
#         train_output = model(image_in)

  

#         label_in=label_in.view(-1, 1).type_as(train_output)

      
#         loss = criterion(train_output, label_in)
        
#         prediction = torch.sigmoid(train_output)



#         loss.backward()

#         optimizer.step()
        
        
#         batchloss = loss.item()
#         t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
#         train_pred.append(prediction.detach().cpu().numpy())

#         train_truth.append(label_in.cpu().numpy())

#         epoch_loss += batchloss
#         epoch_acc += t_acc
#         count = i
        
#         if i % 100 == 99:

#           print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

#       train_pred = stack(train_pred)
#       train_truth = stack(train_truth)
#       print("epoch auc: ", roc_auc_score(train_truth, train_pred))
#       print("epoch average loss: ", epoch_loss/(len(for_train)))

#       train_roc_auc.append(roc_auc_score(train_truth, train_pred))
#       train_loss.append(epoch_loss/(len(for_train)))

#       model.eval()
      
#       val_epoch_loss = 0.0
#       val_epoch_acc = 0.0  
#       with torch.no_grad():
#         for j, dat in enumerate(dataloadingforval, 0):

#           v_image_in, v_label_in = dat
          
          
        

#           v_label_in= v_label_in.to(device)

#           v_image_in= v_image_in.to(device)

          
          
#           val_output = model(v_image_in)


#           loss = criterion(val_output, v_label_in)

#           v_prediction = torch.sigmoid(val_output)
          

          
#           batchloss = loss.item()


#           final_prediction.append(v_prediction.detach().cpu().numpy())

#           ground_truth.append(v_label_in.cpu().numpy())

          


          

#           if j % 10 == 9:


#             print("this is ", j+1, "th patch")


#           val_epoch_loss += batchloss
    

#         final_prediction = stack(final_prediction)

#         ground_truth = stack(ground_truth)


#         print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



#         print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

#         scheduler.step(roc_auc_score(ground_truth, final_prediction))
#         val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
#         val_loss.append(val_epoch_loss/(len(for_val))) 

#   torch.save(model, "/content/model/EfficientNetB4_folderNo._{}.pt".format(time))
#   torch.cuda.empty_cache()

#   # Fine-tune ResNet CNNs
#   print('Fine-tune ResNet CNNs')
#   print("this is folder No. ", time)
#   learning_rate = 0.001
#   epochs = 10
#   model = RES
#   model = model.to(device)

#   train_roc_auc = []

#   train_loss = []

#   val_roc_auc = []

#   val_loss = []





#   optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



#   criterion = nn.BCEWithLogitsLoss()
#   scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#           optimizer,
#           patience=1,
#           factor=0.2,
#           threshold=0.001,
#           mode="max",
#           verbose=True
#       )





#   for epoch in range(epochs):
#       train_pred = []
#       train_truth=[]
      
#       final_prediction= []
#       ground_truth = []
#       print("this is epoch ", epoch+1, "running")
      
#       epoch_loss=0.0    
#       epoch_acc =0.0
      
#       #train the model
#       model.train()
      
#       batch_iteration = 0
#       for i, data in enumerate(dataloadingfortrain, 0):

#         image_in, label_in = data
#         optimizer.zero_grad()
  
        
#         label_in= label_in.to(device)

#         image_in= image_in.to(device)
      
        
#         train_output = model(image_in)

  

#         label_in=label_in.view(-1, 1).type_as(train_output)

      
#         loss = criterion(train_output, label_in)
        
#         prediction = torch.sigmoid(train_output)



#         loss.backward()

#         optimizer.step()
        
        
#         batchloss = loss.item()
#         t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
#         train_pred.append(prediction.detach().cpu().numpy())

#         train_truth.append(label_in.cpu().numpy())

#         epoch_loss += batchloss
#         epoch_acc += t_acc
#         count = i
        
#         if i % 100 == 99:

#           print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

#       train_pred = stack(train_pred)
#       train_truth = stack(train_truth)
#       print("epoch auc: ", roc_auc_score(train_truth, train_pred))
#       print("epoch average loss: ", epoch_loss/(len(for_train)))

#       train_roc_auc.append(roc_auc_score(train_truth, train_pred))
#       train_loss.append(epoch_loss/(len(for_train)))

#       model.eval()
      
#       val_epoch_loss = 0.0
#       val_epoch_acc = 0.0  
#       with torch.no_grad():
#         for j, dat in enumerate(dataloadingforval, 0):

#           v_image_in, v_label_in = dat
          
          
        

#           v_label_in= v_label_in.to(device)

#           v_image_in= v_image_in.to(device)

          
          
#           val_output = model(v_image_in)


#           loss = criterion(val_output, v_label_in)

#           v_prediction = torch.sigmoid(val_output)
          

          
#           batchloss = loss.item()


#           final_prediction.append(v_prediction.detach().cpu().numpy())

#           ground_truth.append(v_label_in.cpu().numpy())

          


          

#           if j % 10 == 9:


#             print("this is ", j+1, "th patch")


#           val_epoch_loss += batchloss
    

#         final_prediction = stack(final_prediction)

#         ground_truth = stack(ground_truth)


#         print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



#         print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

#         scheduler.step(roc_auc_score(ground_truth, final_prediction))
#         val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
#         val_loss.append(val_epoch_loss/(len(for_val))) 


#   torch.save(model, "/content/model/ResNet50_folderNo._{}.pt".format(time))

# print("pretrained CNN tuning complete")



In [ ]:
# id = '1yE-5rmrWIjG4xiO3Stsj2pAG63NUoQxk'
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('train.csv')

##2.2 Training using EfficientB4 + MLP

In [ ]:
#MLP:

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()



        self.predictor = nn.Sequential(
                
                nn.Linear(1793*pic,1024),
                #nn.LayerNorm([1,1024]),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(1024,512),
                #nn.LayerNorm([1,512]),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                #nn.LayerNorm([1,256]),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,1)
                
            )

    def forward(self, x):
              
        z = self.predictor(x)
        z = torch.flatten(z)
        z = z.view(-1,1)
        return z

In [ ]:
# Making prediction and extracting image features

class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON

    self.sig = nn.Sigmoid()

    
    
    


  def forward(self, image):
    feature_record=[]
    feature_record2=[]


    for img in image:
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)


      feature2 = torch.flatten(feature2,1)

      feature = self.sig(feature)

    


      feature_shape = feature.shape[1]

      feature_shape2 = feature2.shape[1]


 
      feature_record.append(feature)
      feature_record2.append(feature2)
   

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)


    
    
    return feature_record,feature_record2

In [ ]:
# Define data loading for patient-level classification

class CustomImageDataset3(Dataset):
    def __init__(self, patient_list, img_dir, transform=None, target_transform=None):
        self.patient_list = patient_list
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
      total = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]].tolist()
      malignant = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]][full_train["target"] == 1].tolist()
      temp = full_train["image_name"][full_train["image_name"].isin(malignant) == False][full_train["patient_id"]== self.patient_list[idx]].tolist()
      img_list =[]
      if len(malignant) >= pic:
        
        img_list.extend(malignant[0:pic])
      else:
        if len(temp)>= pic-len(malignant):
          img_list.extend(malignant)
          
          img_list.extend(temp[0:pic-len(malignant)])
        else:
          img_list.extend(temp)
          for i in range(pic-len(malignant)-len(temp)):
            img_list.extend([total[0]])


      if len(img_list)!=pic:
        for i in range(pic-len(img_list)):
          img_list.extend([total[0]])

     
      img_labels = [full_train["target"][full_train["image_name"]== a].values[0] for a in img_list]
      img_labels = torch.Tensor(img_labels)

      if len(malignant)>0:
        pat_label = torch.tensor([1], dtype= torch.float32) 
      else:
        pat_label = torch.tensor([0], dtype= torch.float32)

      img_paths = [os.path.join(self.img_dir, i +".png") for i in img_list]
      images = [cv2.imread(img_path) for img_path in img_paths]

      if self.transform is not None:
          images = [self.transform(image) for image in images]

      
      # print(len(images))
      # print(img_labels.shape)
      
      

      return images, pat_label

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  time +=1
  # if time <5:
  #   continue

  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True
  
  train_patient = []
  val_patient = []  
  
  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')
  # torch.cuda.empty_cache()

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')

  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))

  torch.cuda.empty_cache()

  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  optimizer.zero_grad()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  torch.cuda.empty_cache()

  ## ResNet50
  print('getting ResNet threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)

  torch.cuda.empty_cache()
  #EfficientNet B4 and MLP
  print('train the model EfficientNet B4 and MLP')
  print("this is folder No. ", time)
  pic = 20
  batch_size = 1
  n_class = 1
  batch_size = 1
  n_input = 1793
  n_hidden = 2048
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  train_data_cleaned = CustomImageDataset3(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset3(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)



  # EfficientNetB4 + MLP 

  learning_rate = 0.0001
  epochs = 1
  model1 = Final_model().to(device)

  model = MLP().to(device)

  model1.eval()


  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]


  val_f1 = []

  train_f1 = []



  class_weights = class_weights.to(device)


  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      model.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):
        try:

          image_in, label_in = data
          optimizer.zero_grad()
    
          
          label_in= label_in.to(device)

          
          train_output,train_output2 = model1(image_in)
          train_output[train_output<thres] =torch.tensor([0], dtype= torch.float32).to(device)

          train_output[train_output>=thres] =torch.tensor([1], dtype= torch.float32).to(device)


          train_output = torch.cat((train_output2,train_output), dim =-1)

          train_output =train_output.view(1,-1)
          train_output = model(train_output)



    

          label_in=label_in.view(-1, 1).type_as(train_output)

            
          loss = criterion(train_output, label_in)
          
          prediction = torch.sigmoid(train_output)




          loss.backward()

          optimizer.step()
          
          
          batchloss = loss.item()
          
          t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=True)
          train_pred.append(prediction.detach().cpu().numpy())

          train_truth.append(label_in.cpu().numpy())

          epoch_loss += batchloss
          epoch_acc += t_acc
          count = i
          
          if i % 500 == 499:

            print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)

        except ValueError: 

          print(prediction.detach().cpu().numpy())      

          print(np.round(prediction.detach().cpu().numpy()))           




      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))
      train_roc_auc.append(roc_auc_score(train_truth, train_pred))
      train_loss.append(epoch_loss/(len(train_patient)))
      
      

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in = dat

          
      
          
        
          v_label_in= v_label_in.to(device)


          val_output,val_output2 = model1(v_image_in)



          
          val_output[val_output<thres] =0.0
          val_output[val_output>=thres] =1.0
      
          val_output = torch.cat((val_output2,val_output), dim =-1)

          
          val_output =val_output.view(1,-1)
          val_output = model(val_output)

    
        
      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                
          
          
      
          loss = criterion(val_output, v_label_in)



          v_prediction = torch.sigmoid(val_output)
          
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())


            

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)

      

        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))



        print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(val_patient)))

        scheduler.step(val_epoch_loss/(len(val_patient)))




  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/EfficientNetB4_and_MLP_folderNo._{}.csv".format(time), index = False)
print("cycle complete!")

##2.3 Training with model: EfficientNetB4 + Bidirectional RNN

In [ ]:
#RNN

class Bi_RNN(nn.Module):
    def __init__(self):
        super(Bi_RNN, self).__init__()
 
        self.RNN = nn.RNN(n_input,n_hidden, num_layers = 2, batch_first =True, bidirectional=True)

        self.predictor = nn.Sequential(
                
                nn.Linear(n_hidden*2,1024),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(1024,512),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,n_class)
                
            )

    def forward(self, x):
              
        RNN_out, h_n = self.RNN(x)

        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.predictor(hidden_out)
        z = torch.flatten(z)
        z = z.view(-1,1)
        return z

In [ ]:
# Making prediction and extracting image features

class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON

    self.sig = nn.Sigmoid()

  def forward(self, image):
    feature_record=[]
    feature_record2=[]


    for img in image:
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)

      feature2 = torch.flatten(feature2,1)

      feature = self.sig(feature)

    
      feature_shape = feature.shape[1]

      feature_shape2 = feature2.shape[1]


 
      feature_record.append(feature)
      feature_record2.append(feature2)
   

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)

   
    return feature_record,feature_record2

In [ ]:
# Define data loading for patient-level classification

class CustomImageDataset3(Dataset):
    def __init__(self, patient_list, img_dir, transform=None, target_transform=None):
        self.patient_list = patient_list
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
      total = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]].tolist()
      malignant = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]][full_train["target"] == 1].tolist()
      temp = full_train["image_name"][full_train["image_name"].isin(malignant) == False][full_train["patient_id"]== self.patient_list[idx]].tolist()
      img_list =[]
      if len(malignant) >= pic:
        
        img_list.extend(malignant[0:pic])
      else:
        if len(temp)>= pic-len(malignant):
          img_list.extend(malignant)
          
          img_list.extend(temp[0:pic-len(malignant)])
        else:
          img_list.extend(temp)
          for i in range(pic-len(malignant)-len(temp)):
            img_list.extend([total[0]])


      if len(img_list)!=pic:
        for i in range(pic-len(img_list)):
          img_list.extend([total[0]])

     
      img_labels = [full_train["target"][full_train["image_name"]== a].values[0] for a in img_list]
      img_labels = torch.Tensor(img_labels)

      if len(malignant)>0:
        pat_label = torch.tensor([1], dtype= torch.float32) 
      else:
        pat_label = torch.tensor([0], dtype= torch.float32)

      img_paths = [os.path.join(self.img_dir, i +".png") for i in img_list]
      images = [cv2.imread(img_path) for img_path in img_paths]

      if self.transform is not None:
          images = [self.transform(image) for image in images]

      
      # print(len(images))
      # print(img_labels.shape)
      
      

      return images, pat_label

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  time +=1  
  # if time < 5:
  #   continue
  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True

  train_patient = []
  val_patient = []  
  
  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')

  # CON = torch.load('EfficientNetB4_pretrained_final.pt')

  # RESNET = torch.load('ResNet50_pretrained_Final.pt')

  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))

  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  
  ## ResNet50
  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)

  pic = 20
  batch_size = 1
  n_class = 1
  batch_size = 1
  n_input = 1793
  n_hidden = 2048
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  train_data_cleaned = CustomImageDataset3(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset3(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)

  print("Training model EfficientNetB4 + Bi-directional RNN  5 images per patient")
  print("this is folder No. ", time)
  # EfficientNetB4 + Bi-directional RNN  5 images per patient

  learning_rate = 0.001
  epochs = 1
  model1 = Final_model().to(device)
  # model2 = Final_model1().to(device)
  model = Bi_RNN().to(device)

  model1.eval()

  # model2.eval()
  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]


  val_f1 = []

  train_f1 = []


  # pos_weight = class_weights
  class_weights = class_weights.to(device)
  # weight=class_weights,reduction='mean'

  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss(weight=class_weights,reduction='mean')
  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      model.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):
        try:

          image_in, label_in = data
          optimizer.zero_grad()
    
          
          label_in= label_in.to(device)

          
          train_output,train_output2 = model1(image_in)
          train_output[train_output<thres] =torch.tensor([0], dtype= torch.float32).to(device)

          train_output[train_output>=thres] =torch.tensor([1], dtype= torch.float32).to(device)

          train_output = torch.cat((train_output2,train_output), dim =-1)

          train_output = model(train_output)



    

          label_in=label_in.view(-1, 1).type_as(train_output)

            
          loss = criterion(train_output, label_in)
          
          prediction = torch.sigmoid(train_output)




          loss.backward()

          optimizer.step()
          
          
          batchloss = loss.item()
          
          t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=True)
          train_pred.append(prediction.detach().cpu().numpy())

          train_truth.append(label_in.cpu().numpy())

          epoch_loss += batchloss
          epoch_acc += t_acc
          count = i
          
          if i % 500 == 499:

            print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)

        except ValueError: 

          print(prediction.detach().cpu().numpy())      

          print(np.round(prediction.detach().cpu().numpy()))           




      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))
      train_roc_auc.append(roc_auc_score(train_truth, train_pred))
      train_loss.append(epoch_loss/(len(train_patient)))
      
      

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in = dat

          
          
        
          v_label_in= v_label_in.to(device)


          val_output,val_output2 = model1(v_image_in)



          
          val_output[val_output<thres] =0.0
          val_output[val_output>=thres] =1.0

          val_output = torch.cat((val_output2,val_output), dim =-1)




          val_output = model(val_output)

    
        
      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                
      
          loss = criterion(val_output, v_label_in)



          v_prediction = torch.sigmoid(val_output)
          
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

    

    

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)

        

        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))

        print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(val_patient)))

        scheduler.step(val_epoch_loss/(len(val_patient)))




  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/EfficientNetB4_and_BIRNN_folderNo._{}.csv".format(time), index = False)

this is folder No.  1
getting EfficientNet threshold
this is folder No.  1
this is  100 th patch
this is  200 th patch
this is  300 th patch
this is  400 th patch
this is  500 th patch
this is  600 th patch
this is  700 th patch
this is  800 th patch
epoch average loss for validation:  0.0090157428786986
validation roc_auc_score 0.9041654789242499
getting EfficientNet threshold
this is folder No.  1
this is  100 th patch
this is  200 th patch
this is  300 th patch
this is  400 th patch
this is  500 th patch
this is  600 th patch
this is  700 th patch
this is  800 th patch
epoch average loss for validation:  0.011058334548536523
validation roc_auc_score 0.792006952320247
Training model EfficientNetB4 + Bi-directional RNN  5 images per patient
this is folder No.  1
this is epoch  1 running
epoch  1   500 th batch accuracy:  0.0 loss:  4.111856460571289


##2.4 Training with model: EfficientNetB4 + Bi-directional LSTM

In [ ]:
# LSTM 

class Bi_LSTM(nn.Module):
    def __init__(self):
        super(Bi_LSTM, self).__init__()
 
        self.lstm = nn.LSTM(n_input,n_hidden, num_layers = 2, batch_first =True, bidirectional=True)

        self.predictor = nn.Sequential(
                
                nn.Linear(n_hidden*2,1024),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(1024,512),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,n_class)
                
            )

    def forward(self, x):
              
        lstm_out, (h_n,c_n) = self.lstm(x)

        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.predictor(hidden_out)
        z = torch.flatten(z)
        z = z.view(-1,1)
        return z

In [ ]:
class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON

    self.sig = nn.Sigmoid()

    
    
    


  def forward(self, image):
    feature_record=[]
    feature_record2=[]


    for img in image:
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)


      feature2 = torch.flatten(feature2,1)

      feature = self.sig(feature)
 





      feature_shape = feature.shape[1]

      feature_shape2 = feature2.shape[1]

 
      feature_record.append(feature)
      feature_record2.append(feature2)


    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)

    
    
    return feature_record,feature_record2

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  time +=1
  # if time <5:
  #   continue
  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True

  train_patient = []
  val_patient = []  
  
  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')

  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))


  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  
  ## ResNet50
  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)

  pic = 15
  batch_size = 1
  n_class = 1
  batch_size = 1
  n_input = 1793
  n_hidden = 2048
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  train_data_cleaned = CustomImageDataset3(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset3(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # EfficientNetB4 + Bi-directional LSTM
  print("Training model EfficientNetB4 + Bi-directional LSTM 5 PIC")
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model1 = Final_model().to(device)

  model = Bi_LSTM().to(device)

  model1.eval()


  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]


  val_f1 = []

  train_f1 = []



  class_weights = class_weights.to(device)

  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      model.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):
        try:

          image_in, label_in = data
          optimizer.zero_grad()
    
          
          label_in= label_in.to(device)

          
          train_output,train_output2 = model1(image_in)
          train_output[train_output<thres] =torch.tensor([0], dtype= torch.float32).to(device)

          train_output[train_output>=thres] =torch.tensor([1], dtype= torch.float32).to(device)

    
          train_output = torch.cat((train_output2,train_output), dim =-1)

    
          train_output = model(train_output)



    

          label_in=label_in.view(-1, 1).type_as(train_output)

            
          loss = criterion(train_output, label_in)
          
          prediction = torch.sigmoid(train_output)




          loss.backward()

          optimizer.step()
          
          
          batchloss = loss.item()
          
          t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=True)
          train_pred.append(prediction.detach().cpu().numpy())

          train_truth.append(label_in.cpu().numpy())

          epoch_loss += batchloss
          epoch_acc += t_acc
          count = i
          
          if i % 500 == 499:

            print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)

        except ValueError: 

          print(prediction.detach().cpu().numpy())      

          print(np.round(prediction.detach().cpu().numpy()))           




      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))
      train_roc_auc.append(roc_auc_score(train_truth, train_pred))
      train_loss.append(epoch_loss/(len(train_patient)))
      
      

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in = dat

          

        
          v_label_in= v_label_in.to(device)


          val_output,val_output2 = model1(v_image_in)



          
          val_output[val_output<thres] =0.0
          val_output[val_output>=thres] =1.0

          val_output = torch.cat((val_output2,val_output), dim =-1)





          

          val_output = model(val_output)

    
        
      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                
          
          
      
          loss = criterion(val_output, v_label_in)



          v_prediction = torch.sigmoid(val_output)
          
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())


          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        

        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))



        print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(val_patient)))

        scheduler.step(val_epoch_loss/(len(val_patient)))




  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/EfficientNetB4_and_BILSTM_folderNo._{}.csv".format(time), index = False)

#Section 3 Training using image feature and personal information of patients (gender,age etc.)

##3.1 Dataloading set up & hyperparameter control

In [ ]:
# Data loading with personal information for patient-level classification


class CustomImageDataset2(Dataset):
    def __init__(self, patient_list, img_dir, transform=None, target_transform=None):
        self.patient_list = patient_list
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
      total = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]].tolist()
      malignant = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]][full_train["target"] == 1].tolist()
      temp = full_train["image_name"][full_train["image_name"].isin(malignant) == False][full_train["patient_id"]== self.patient_list[idx]].tolist()
      img_list =[]
      if len(malignant) >=pic:
        
        img_list.extend(malignant[0:pic])
      else:
        if len(temp)>= pic-len(malignant):
          img_list.extend(malignant)
          
          img_list.extend(temp[0:pic-len(malignant)])
        else:
          img_list.extend(temp)
          for i in range(pic-len(malignant)-len(temp)):
            img_list.extend([total[0]])


      if len(img_list)!=pic:
        for i in range(pic-len(img_list)):
          img_list.extend([total[0]])

        
      img_labels = full_train["target"][full_train["image_name"].isin(img_list)].tolist()


      if len(malignant)> 0:
        pat_label = torch.tensor([1], dtype= torch.float32) 
      else:
        pat_label = torch.tensor([0], dtype= torch.float32)


      patient_condition=[]
      patient_age = [full_train["age_group"][full_train["image_name"] == im] for im in img_list]
      patient_sex = [full_train["sex"][full_train["image_name"]== im] for im in img_list]
      lesion_location = [full_train["anatom_site_general_challenge"][full_train["image_name"]== im] for im in img_list]
      patient_age = np.array(onehot(patient_age,10))
      patient_sex = np.array(onehot(patient_sex,3))
      lesion_location = np.array(onehot(lesion_location,6))

      patient_condition= np.concatenate((patient_age,patient_sex),1)
      patient_condition= np.concatenate((patient_condition,lesion_location),1)
      if len(patient_condition)!=pic:
        use = np.zeros(19)
      for i in range(pic-len(patient_condition)):
        patient_condition=np.vstack((patient_condition, use))



      img_paths = [os.path.join(self.img_dir, i +".png") for i in img_list]
      images = [cv2.imread(img_path) for img_path in img_paths]

      if self.transform is not None:
          images = [self.transform(image) for image in images]

      return images, pat_label, patient_condition

##3.2 Training using model: EfficientNetB4 + Bi-directional LSTM

In [ ]:
# LSTM 

class Bi_LSTM(nn.Module):
    def __init__(self):
        super(Bi_LSTM, self).__init__()
 
        self.lstm = nn.LSTM(n_input,n_hidden, num_layers = 2, batch_first =True, bidirectional=True)

        self.predictor = nn.Sequential(
                
                nn.Linear(n_hidden*2,1024),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(1024,512),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,n_class)
                
            )

    def forward(self, x):
              
        lstm_out, (h_n,c_n) = self.lstm(x)

        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.predictor(hidden_out)
        z = torch.flatten(z)
        z = z.view(-1,1)
        return z

In [ ]:
# combine personal information with image prediction and feature.

class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON


    self.sig = nn.Sigmoid()
    
    
    


  def forward(self, image, metadata):
    feature_record=[]
    feature_record2=[]

    for img, meta in zip(image,metadata):
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)
 

      feature2 = torch.flatten(feature2,1)

  

      feature = self.sig(feature)

      feature = feature.view(-1, 1)
      meta = meta.view(-1,19)

      feature2 = torch.cat((feature2,meta),1)

      
      feature_shape = feature.shape[1]
      feature_record.append(feature)
      feature_shape2 = feature2.shape[1]
      feature_record2.append(feature2)
      

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)
    
    
    return feature_record, feature_record2

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):

  time +=1
  # if time <3:
  #   continue
  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True

  train_patient = []
  val_patient = []  
  
  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')

  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))

  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  
  ## ResNet50
  print('getting ResNet50 threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)


  batch_size = 1
  pic = 20
  train_data_cleaned = CustomImageDataset2(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset2(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)
  n_class = 1
  batch_size =1
  n_input = 1793+19
  n_hidden = 2048

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



  # EfficientNetB4 + Bi-directional LSTM with meta info 5 PIC
  print("Training model EfficientNetB4 + Bi-directional LSTM with meta info 5 PIC")
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model1 = Final_model().to(device)

  model = Bi_LSTM().to(device)

  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]

  val_f1 = []

  train_f1 = []


  class_weights = class_weights.to(device)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )

  model1.eval()



  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      model.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):

        image_in, label_in, meta = data
        optimizer.zero_grad()
  
        
        label_in= label_in.to(device)

        meta= meta.to(device)
      
        meta= meta.view(pic,-1,19)
        

        pred, out = model1(image_in, meta)
    
        pred[pred<thres] =0
        pred[pred>=thres] =1
        train_output = torch.cat((out,pred), dim =-1)
        train_output = model(train_output.float())




        label_in=label_in.view(-1, 1).type_as(train_output)
  
      
        loss = criterion(train_output, label_in)
        
        prediction = torch.sigmoid(train_output)




        loss.backward()

        optimizer.step()
        
        
        batchloss = loss.item()
        t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
        train_pred.append(prediction.detach().cpu().numpy())

        train_truth.append(label_in.cpu().numpy())

        epoch_loss += batchloss
        epoch_acc += t_acc
        count = i
        
        if i % 500 == 499:

          print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)
          



      

      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))

      train_roc_auc.append(roc_auc_score(train_truth, train_pred))

      train_loss.append(epoch_loss/(len(train_patient)))

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in, v_meta = dat
          
          


          v_label_in= v_label_in.to(device)

          v_meta= v_meta.to(device)

          v_meta =v_meta.view(pic,-1,19)
          
  
          pred, out = model1(v_image_in, v_meta)
          pred[pred<thres] =0
          pred[pred>=thres] =1
          val_output = torch.cat((out,pred), dim =-1)
          val_output = model(val_output.float())

      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

      

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))



      print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))

      val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))

      val_loss.append(val_epoch_loss/(len(val_patient)))


      scheduler.step(val_epoch_loss/(len(val_patient))) 





  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/EfficientNetB4_and_BILSTM_with_metainfo_folderNo._{}.csv".format(time), index = False)

##3.3 Training using model: EfficientNetB4 + Bi-directional LSTM with Attention

In [ ]:
## Bi-LSTM with attention mechanism

class Bi_LSTM_attention(nn.Module):
    def __init__(self, n_input, n_hidden):
        super(Bi_LSTM_attention, self).__init__()

        self.ATTN_TYPE_DOT_PRODUCT = "Dot Product"
        self.ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"
        self.ATTN_TYPE_GENERAL_DOT_PRODUCT = "General Dot Product"
        self.n_input=n_input
        self.n_hidden= n_hidden
        self.dropout = nn.Dropout(0.5)
        
        self.lstm_encoder = nn.LSTM(n_input, n_hidden // 2,
                                  num_layers=1,batch_first =True, bidirectional=True)
        
        self.lstm_decoder = nn.LSTM(n_input, n_hidden // 2,
                                  num_layers=1,batch_first =True, bidirectional=True)
        
        
        self.hidden = self.init_hidden()
        

        self.predictor = nn.Sequential(
                nn.Linear(self.n_hidden*pic*2,512),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,128),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(128,n_class)
                
            )


    def init_hidden(self):
      return (torch.randn(2, 1, self.n_hidden // 2).to(device),
              torch.randn(2, 1, self.n_hidden // 2).to(device))
    
  
    
    def _cal_attention(self, hidden, encoder_hiddens, method):

        if method == self.ATTN_TYPE_DOT_PRODUCT:
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)


        elif method == self.ATTN_TYPE_SCALE_DOT_PRODUCT:
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T)/np.sqrt(self.n_hidden),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)

        elif method == self.ATTN_TYPE_GENERAL_DOT_PRODUCT:
          self.att_weight = nn.Parameter(torch.randn(hidden.squeeze(1).shape[1],hidden.squeeze(1).shape[1]))
          hidden = torch.matmul(hidden.squeeze(1), self.att_weight).unsqueeze(1)
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)
        
        return concat_output



    def forward(self, x):
      self.hidden = self.init_hidden()
      encoder_out, self.hidden  = self.lstm_encoder(x, self.hidden) 
      (h_n, c_n) = self.hidden
      lstm_out, self.hidden  = self.lstm_decoder(x, self.hidden)
      hidden_out = torch.cat((h_n[0,:,:],h_n[1,:,:]),1)     
      attention_output = self._cal_attention(lstm_out, hidden_out,self.ATTN_TYPE_DOT_PRODUCT)

      attention_output = attention_output.view(batch_size, self.n_hidden*pic*2)
      attention_output = self.dropout(attention_output)
                

      z = self.predictor(attention_output)
      return z

In [ ]:
# combine personal information with image prediction and feature

class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON



    self.sig = nn.Sigmoid()
 

  def forward(self, image, metadata):
    feature_record=[]
    feature_record2=[]

    for img, meta in zip(image,metadata):
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)


      feature2 = torch.flatten(feature2,1)

  

      feature = self.sig(feature)

      feature = feature.view(-1, 1)
      meta = meta.view(-1,19)

      feature2 = torch.cat((feature2,meta),1)

      
      feature_shape = feature.shape[1]
      feature_record.append(feature)
      feature_shape2 = feature2.shape[1]
      feature_record2.append(feature2)
      

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)


    
    
    return feature_record, feature_record2

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  time +=1
  # if time < 5:
  #   continue
  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True

  train_patient = []
  val_patient = []  
  
  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')

  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))

  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  
  ## ResNet50
  print('getting ResNet50 threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)
  # EfficientNetB4 and Bi-directional LSTM
  print("Training model EfficientNetB4 and Bi-directional LSTM")
  print("this is folder No. ", time)

  batch_size = 1
  pic = 20
  train_data_cleaned = CustomImageDataset2(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset2(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)
  n_class = 1
  batch_size =1
  n_input = 1793+19
  n_hidden = 2048

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



  ## EfficientNetB4 + Bi-directional LSTM with Attention with personal information 5 images per patient
  learning_rate = 0.001
  epochs = 1
  model1 = Final_model().to(device)

  model = Bi_LSTM_attention(n_input,n_hidden).to(device)

  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]

  val_f1 = []

  train_f1 = []

  class_weights = class_weights.to(device)

  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )

  model1.eval()




  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      model.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):

        image_in, label_in, meta = data
        optimizer.zero_grad()
  
        
        label_in= label_in.to(device)

        meta= meta.to(device)
      
        meta= meta.view(pic,-1,19)
        
        pred, out = model1(image_in, meta)


        pred[pred<thres] =0
        pred[pred>=thres] =1
        train_output = torch.cat((out,pred), dim =-1)
        train_output = model(train_output.float())



        label_in=label_in.view(-1, 1).type_as(train_output)
  
      
        loss = criterion(train_output, label_in)
        
        prediction = torch.sigmoid(train_output)




        loss.backward()

        optimizer.step()
        
        
        batchloss = loss.item()
        t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
        train_pred.append(prediction.detach().cpu().numpy())

        train_truth.append(label_in.cpu().numpy())

        epoch_loss += batchloss
        epoch_acc += t_acc
        count = i
        
        if i % 500 == 499:

          print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)
          

      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))

      train_roc_auc.append(roc_auc_score(train_truth, train_pred))

      train_loss.append(epoch_loss/(len(train_patient)))
    
      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in, v_meta = dat
          
          


          v_label_in= v_label_in.to(device)

          v_meta= v_meta.to(device)

          v_meta =v_meta.view(pic,-1,19)
          
          pred, out = model1(v_image_in, v_meta)

          pred[pred<thres] =0
          pred[pred>=thres] =1
          val_output = torch.cat((out,pred), dim =-1)
          val_output = model(val_output.float())


      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
        

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))



      print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))

      val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))

      val_loss.append(val_epoch_loss/(len(val_patient)))


      scheduler.step(val_epoch_loss/(len(val_patient))) 




  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/EfficientNetB4_and_BILSTM_Attention_with_metainfo_folderNo._{}.csv".format(time), index = False)

##3.4 Training using EfficientNetB4 + ResNet50 + Bi-directional LSTM with Attention and personal information

In [ ]:
# Data loading with personal information for patient-level classification


class CustomImageDataset2(Dataset):
    def __init__(self, patient_list, img_dir, transform=None, target_transform=None):
        self.patient_list = patient_list
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
      total = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]].tolist()
      malignant = full_train["image_name"][full_train["patient_id"] == self.patient_list[idx]][full_train["target"] == 1].tolist()
      temp = full_train["image_name"][full_train["image_name"].isin(malignant) == False][full_train["patient_id"]== self.patient_list[idx]].tolist()
      img_list =[]
      if len(malignant) >=pic:
        
        img_list.extend(malignant[0:pic])
      else:
        if len(temp)>= pic-len(malignant):
          img_list.extend(malignant)
          
          img_list.extend(temp[0:pic-len(malignant)])
        else:
          img_list.extend(temp)
          for i in range(pic-len(malignant)-len(temp)):
            img_list.extend([total[0]])


      if len(img_list)!=pic:
        for i in range(pic-len(img_list)):
          img_list.extend([total[0]])

        
      img_labels = full_train["target"][full_train["image_name"].isin(img_list)].tolist()


      if len(malignant)> 0:
        pat_label = torch.tensor([1], dtype= torch.float32) 
      else:
        pat_label = torch.tensor([0], dtype= torch.float32)


      patient_condition=[]
      patient_age = [full_train["age_group"][full_train["image_name"] == im] for im in img_list]
      patient_sex = [full_train["sex"][full_train["image_name"]== im] for im in img_list]
      lesion_location = [full_train["anatom_site_general_challenge"][full_train["image_name"]== im] for im in img_list]
      patient_age = np.array(onehot(patient_age,10))
      patient_sex = np.array(onehot(patient_sex,3))
      lesion_location = np.array(onehot(lesion_location,6))

      patient_condition= np.concatenate((patient_age,patient_sex),1)
      patient_condition= np.concatenate((patient_condition,lesion_location),1)
      if len(patient_condition)!=pic:
        use = np.zeros(19)
      for i in range(pic-len(patient_condition)):
        patient_condition=np.vstack((patient_condition, use))



      img_paths = [os.path.join(self.img_dir, i +".png") for i in img_list]
      images = [cv2.imread(img_path) for img_path in img_paths]

      if self.transform is not None:
          images = [self.transform(image) for image in images]

      return images, pat_label, patient_condition

In [ ]:
## Bi-LSTM with Attention Mechanism
class Bi_LSTM_attention(nn.Module):
    def __init__(self, n_input, n_hidden):
        super(Bi_LSTM_attention, self).__init__()

        self.ATTN_TYPE_DOT_PRODUCT = "Dot Product"
        self.ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"
        self.ATTN_TYPE_GENERAL_DOT_PRODUCT = "General Dot Product"
        self.n_input=n_input
        self.n_hidden= n_hidden
        self.dropout = nn.Dropout(0.5)
        
        self.lstm_encoder = nn.LSTM(n_input, n_hidden // 2,
                                  num_layers=1,batch_first =True, bidirectional=True)
        
        self.lstm_decoder = nn.LSTM(n_input, n_hidden // 2,
                                  num_layers=1,batch_first =True, bidirectional=True)
        
        
        self.hidden = self.init_hidden()
        

        self.predictor = nn.Sequential(
                nn.Linear(self.n_hidden*pic*2,512),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(512,256),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(256,128),
                nn.ReLU(),
                nn.Dropout(),
                nn.Linear(128,n_class)
                
            )


    def init_hidden(self):
      return (torch.randn(2, 1, self.n_hidden // 2).to(device),
              torch.randn(2, 1, self.n_hidden // 2).to(device))
    
  
    
    def _cal_attention(self, hidden, encoder_hiddens, method):

        if method == self.ATTN_TYPE_DOT_PRODUCT:
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)


        elif method == self.ATTN_TYPE_SCALE_DOT_PRODUCT:
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T)/np.sqrt(self.n_hidden),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)

        elif method == self.ATTN_TYPE_GENERAL_DOT_PRODUCT:
          self.att_weight = nn.Parameter(torch.randn(hidden.squeeze(1).shape[1],hidden.squeeze(1).shape[1]))
          hidden = torch.matmul(hidden.squeeze(1), self.att_weight).unsqueeze(1)
          attn_weights = F.softmax(torch.matmul(hidden.squeeze(1), encoder_hiddens.squeeze(1).T),dim=-1)
          attn_output = torch.matmul(attn_weights, encoder_hiddens.squeeze(1))
          concat_output = torch.cat((attn_output, hidden.squeeze(1)), -1)
          concat_output = concat_output.unsqueeze(1)
        
        return concat_output



    def forward(self, x):
      self.hidden = self.init_hidden()
      encoder_out, self.hidden  = self.lstm_encoder(x, self.hidden) 
      (h_n, c_n) = self.hidden
      lstm_out, self.hidden  = self.lstm_decoder(x, self.hidden)
      hidden_out = torch.cat((h_n[0,:,:],h_n[1,:,:]),1)     
      attention_output = self._cal_attention(lstm_out, hidden_out,self.ATTN_TYPE_DOT_PRODUCT)

      attention_output = attention_output.view(batch_size, self.n_hidden*pic*2)
      attention_output = self.dropout(attention_output)
                

      z = self.predictor(attention_output)
      return z

In [ ]:
# Set up classification and feature extraction using EfficientB4

class Final_model(nn.Module):

  def __init__(self):

    super(Final_model, self).__init__()
    self.CNN = CON

    self.sig = nn.Sigmoid()
  

  def forward(self, image, metadata):
    feature_record=[]
    feature_record2=[]

    for img, meta in zip(image,metadata):
      
      img = img.to(device)

 
      feature = self.CNN(img)
      feature2= self.CNN.extract_features(img)
      feature2 = self.CNN._avg_pooling(feature2)


      feature2 = torch.flatten(feature2,1)

  

      feature = self.sig(feature)

      feature = feature.view(-1, 1)
      meta = meta.view(-1,19)

      feature2 = torch.cat((feature2,meta),1)

      
      feature_shape = feature.shape[1]
      feature_record.append(feature)
      feature_shape2 = feature2.shape[1]
      feature_record2.append(feature2)
      

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
    feature_record2= torch.stack(feature_record2).reshape(-1,pic,feature_shape2)


    
    
    return feature_record, feature_record2

In [ ]:
# Set up classification and feature extraction using ResNet50

class Final_model1(nn.Module):

  def __init__(self):

    super(Final_model1, self).__init__()
    self.CNN = RESNET
 

    self.sig = nn.Sigmoid()

  
  def forward(self, image):
    feature_record=[]
    feature_record2=[]


    for img in image:
      
      img = img.to(device)

 
      feature = self.CNN(img)
 

      feature = self.sig(feature)
  

      feature_shape = feature.shape[1]

 
      feature_record.append(feature)

        

    feature_record= torch.stack(feature_record).reshape(-1,pic,feature_shape)
  
    
    
    return feature_record

In [ ]:
time = 0
for train_index, test_index in kf.split(Full_patient_list, patient_class):
  time +=1
  # if time <4:
  #   continue
  # CON = EfficientNet.from_pretrained('efficientnet-b4')
  # num_ftrs = CON._fc.in_features
  # CON._fc = nn.Linear(num_ftrs, 1)
  # CON._fc.require_grad = True

  # RES = models.resnet50(pretrained=True)
  # R_num_ftrs =RES.fc.in_features
  # RES.fc = nn.Linear(R_num_ftrs, 1)
  # RES.fc.require_grad = True

  train_patient = []
  val_patient = []  

  print("this is folder No. ", time)
  tep = []
  for i in test_index:
    tep.append(Full_patient_list[i])
    val_patient.append(Full_patient_list[i])
  for_val = full_train[full_train['patient_id'].isin(tep)==True]
  for_train = full_train[full_train['patient_id'].isin(tep)==False]
  for j in train_index:
    train_patient.append(Full_patient_list[j])
  ones = len(for_train[for_train['target']==1])
  zeros = len(for_train)-ones
  a = np.zeros(zeros)
  b = np.ones(ones)
  c = np.concatenate((a,b))
  random.shuffle(c)

  class_weights=sklearn.utils.class_weight.compute_class_weight('balanced',classes = np.unique(c),y=c)
  class_weights=torch.tensor(class_weights,dtype=torch.float)
  class_weights = class_weights[1]/class_weights[0]

  batch_size = 30
  train_data_cleaned = CustomImageDataset(for_train, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset(for_val, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)


  # # Fine-tune efficientNet CNNs
  # print('Fine-tune efficientNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = CON
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 

  # torch.save(model, 'EfficientNetB4_pretrained_final.pt')

  # # Fine-tune ResNet CNNs
  # print('Fine-tune ResNet CNNs')
  # print("this is folder No. ", time)
  # learning_rate = 0.001
  # epochs = 10
  # model = RES
  # model = model.to(device)

  # train_roc_auc = []

  # train_loss = []

  # val_roc_auc = []

  # val_loss = []





  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  # criterion = nn.BCEWithLogitsLoss()
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  #         optimizer,
  #         patience=1,
  #         factor=0.2,
  #         threshold=0.001,
  #         mode="max",
  #         verbose=True
  #     )





  # for epoch in range(epochs):
  #     train_pred = []
  #     train_truth=[]
      
  #     final_prediction= []
  #     ground_truth = []
  #     print("this is epoch ", epoch+1, "running")
      
  #     epoch_loss=0.0    
  #     epoch_acc =0.0
      
  #     #train the model
  #     model.train()
      
  #     batch_iteration = 0
  #     for i, data in enumerate(dataloadingfortrain, 0):

  #       image_in, label_in = data
  #       optimizer.zero_grad()
  
        
  #       label_in= label_in.to(device)

  #       image_in= image_in.to(device)
      
        
  #       train_output = model(image_in)

  

  #       label_in=label_in.view(-1, 1).type_as(train_output)

      
  #       loss = criterion(train_output, label_in)
        
  #       prediction = torch.sigmoid(train_output)



  #       loss.backward()

  #       optimizer.step()
        
        
  #       batchloss = loss.item()
  #       t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
  #       train_pred.append(prediction.detach().cpu().numpy())

  #       train_truth.append(label_in.cpu().numpy())

  #       epoch_loss += batchloss
  #       epoch_acc += t_acc
  #       count = i
        
  #       if i % 100 == 99:

  #         print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)





      

  #     train_pred = stack(train_pred)
  #     train_truth = stack(train_truth)
  #     print("epoch auc: ", roc_auc_score(train_truth, train_pred))
  #     print("epoch average loss: ", epoch_loss/(len(for_train)))

  #     train_roc_auc.append(roc_auc_score(train_truth, train_pred))
  #     train_loss.append(epoch_loss/(len(for_train)))

  #     model.eval()
      
  #     val_epoch_loss = 0.0
  #     val_epoch_acc = 0.0  
  #     with torch.no_grad():
  #       for j, dat in enumerate(dataloadingforval, 0):

  #         v_image_in, v_label_in = dat
          
          
        

  #         v_label_in= v_label_in.to(device)

  #         v_image_in= v_image_in.to(device)

          
          
  #         val_output = model(v_image_in)


  #         loss = criterion(val_output, v_label_in)

  #         v_prediction = torch.sigmoid(val_output)
          

          
  #         batchloss = loss.item()


  #         final_prediction.append(v_prediction.detach().cpu().numpy())

  #         ground_truth.append(v_label_in.cpu().numpy())

          


          

  #         if j % 10 == 9:


  #           print("this is ", j+1, "th patch")


  #         val_epoch_loss += batchloss
    

  #       final_prediction = stack(final_prediction)

  #       ground_truth = stack(ground_truth)


  #       print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



  #       print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

  #       scheduler.step(roc_auc_score(ground_truth, final_prediction))
  #       val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
  #       val_loss.append(val_epoch_loss/(len(for_val))) 


  # torch.save(model, 'ResNet50_pretrained_Final.pt')


  CON = torch.load("/content/model/EfficientNetB4_folderNo._{}.pt".format(time))

  RESNET = torch.load("/content/model/ResNet50_folderNo._{}.pt".format(time))



  print('getting EfficientNet threshold')
  print("this is folder No. ", time)
  ## EfficientNetB4
  learning_rate = 0.001
  epochs = 1
  model = CON
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
          
          
        

          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)

          
          
          val_output = model(v_image_in)


          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

          


          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val))) 

  thres, _ = threshold(ground_truth,final_prediction)

  
  ## ResNet50
  print('getting ResNet50 threshold')
  print("this is folder No. ", time)
  learning_rate = 0.001
  epochs = 1
  model = RESNET
  model = model.to(device)

  train_roc_auc = []

  train_loss = []

  val_roc_auc = []

  val_loss = []




  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)
  #criterion = nn.CrossEntropyLoss()
  criterion = nn.BCEWithLogitsLoss()
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=1,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )





  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
  

      model.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingfortrain, 0):

          v_image_in, v_label_in = dat
                         
          v_label_in= v_label_in.to(device)

          v_image_in= v_image_in.to(device)          
          
          val_output = model(v_image_in)

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          
          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())
          

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)


        print("epoch average loss for validation: ", val_epoch_loss/(len(for_val)))



        print("validation roc_auc_score", roc_auc_score(ground_truth, final_prediction))

        scheduler.step(roc_auc_score(ground_truth, final_prediction))
        val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))
        val_loss.append(val_epoch_loss/(len(for_val)))

  thres2, _ = threshold(ground_truth,final_prediction)

  # EfficientNetB4 and ResNet50 and Bi-directional LSTM with Attention and personal info

  print("Training model EfficientNetB4 and ResNet50 and Bi-directional LSTM with Attention and personal info")
  print("this is folder No. ", time)
  batch_size = 1
  pic = 15
  train_data_cleaned = CustomImageDataset2(train_patient, '/content/train/', train_dat_tran)
  val_data_cleaned = CustomImageDataset2(val_patient, '/content/train/', val_dat_tran)
  dataloadingfortrain = DataLoader(train_data_cleaned, batch_size = batch_size, shuffle = True)
  dataloadingforval = DataLoader(val_data_cleaned, batch_size = batch_size, shuffle = True)
  # Here we add extra one to n_input, this is the extra classification made by ResNet 50

  n_class = 1
  batch_size =1
  n_input = 1793+20
  n_hidden = 2048

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  ## Training using EfficientNetB4 + ResNet50 + Bi-directional LSTM with Attention and personal information 5 images per patient
  print("training ultimate model")
  learning_rate = 0.001
  epochs = 1
  model1 = Final_model().to(device)
  model2 = Final_model1().to(device)
  LSTM = Bi_LSTM_attention(n_input,n_hidden).to(device)

  train_roc_auc =[]

  train_loss=[]

  val_roc_auc=[]

  val_loss=[]

  val_f1 = []

  train_f1 = []

  class_weights = class_weights.to(device)

  #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)

  #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
  #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(LSTM.parameters(), lr=learning_rate, weight_decay=1e-4)
  criterion = nn.BCEWithLogitsLoss(pos_weight = class_weights)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
          optimizer,
          patience=2,
          factor=0.2,
          threshold=0.001,
          mode="max",
          verbose=True
      )

  model1.eval()
  model2.eval()



  for epoch in range(epochs):
      train_pred = []
      train_truth=[]
      
      final_prediction= []
      ground_truth = []
      print("this is epoch ", epoch+1, "running")
      
      epoch_loss=0.0    
      epoch_acc =0.0
      
      #train the model
      LSTM.train()
      
      batch_iteration = 0
      for i, data in enumerate(dataloadingfortrain, 0):

        image_in, label_in, meta = data
        optimizer.zero_grad()
  
        
        label_in= label_in.to(device)

        meta= meta.to(device)
      
        meta= meta.view(pic,-1,19)
        
        pred, out = model1(image_in, meta)

        pred2 = model2(image_in)      
        pred[pred<thres] =0
        pred[pred>=thres] =1
        pred2[pred2<thres2] =0
        pred2[pred2>=thres2] =1
        pred = torch.cat((pred,pred2),dim=-1)
        train_output = torch.cat((out,pred), dim =-1)
        train_output = LSTM(train_output.float())




        label_in=label_in.view(-1, 1).type_as(train_output)
  
      
        loss = criterion(train_output, label_in)
        
        prediction = torch.sigmoid(train_output)




        loss.backward()

        optimizer.step()
        
        # teet = np.any(np.isnan(np.round(prediction.detach().cpu().numpy())))
        # if teet == True:
        #   print(np.round(prediction.detach().cpu().numpy()))
        #   print(prediction.detach().cpu().numpy())
        batchloss = loss.item()
        t_acc = accuracy_score(label_in.cpu().numpy() , np.round(prediction.detach().cpu().numpy()),normalize=False)
        train_pred.append(prediction.detach().cpu().numpy())

        train_truth.append(label_in.cpu().numpy())

        epoch_loss += batchloss
        epoch_acc += t_acc
        count = i
        
        if i % 500 == 499:

          print("epoch ", epoch + 1," ", i + 1,"th batch", "accuracy: ", t_acc, "loss: ", batchloss)
          


      train_pred = stack(train_pred)
      train_truth = stack(train_truth)
      print("epoch average auc: ", roc_auc_score(train_truth, train_pred))
      print("epoch average loss: ", epoch_loss/(len(train_patient)))

      train_roc_auc.append(roc_auc_score(train_truth, train_pred))

      train_loss.append(epoch_loss/(len(train_patient)))
  
      LSTM.eval()
      
      val_epoch_loss = 0.0
      val_epoch_acc = 0.0  
      with torch.no_grad():
        for j, dat in enumerate(dataloadingforval, 0):

          v_image_in, v_label_in, v_meta = dat
          
          


          v_label_in= v_label_in.to(device)

          v_meta= v_meta.to(device)

          v_meta =v_meta.view(pic,-1,19)
          
  
          pred, out = model1(v_image_in, v_meta)
          pred2 = model2(v_image_in)  
          pred[pred<thres] =0
          pred[pred>=thres] =1

          pred2[pred2<thres2] =0
          pred2[pred2>=thres2] =1
          pred = torch.cat((pred,pred2),dim=-1)

          val_output = torch.cat((out,pred), dim =-1)
          val_output = LSTM(val_output.float())

      
          v_label_in=v_label_in.view(-1, 1).type_as(val_output)
                

          loss = criterion(val_output, v_label_in)

          v_prediction = torch.sigmoid(val_output)
          

          
          batchloss = loss.item()


          final_prediction.append(v_prediction.detach().cpu().numpy())

          ground_truth.append(v_label_in.cpu().numpy())

        

          if j % 100 == 99:


            print("this is ", j+1, "th patch")


          val_epoch_loss += batchloss
    

        final_prediction = stack(final_prediction)

        ground_truth = stack(ground_truth)
      


        print("epoch average loss for validation: ", val_epoch_loss/(len(val_patient)))



      print("roc_auc_score", roc_auc_score(ground_truth, final_prediction))

      val_roc_auc.append(roc_auc_score(ground_truth, final_prediction))

      val_loss.append(val_epoch_loss/(len(val_patient)))


      scheduler.step(val_epoch_loss/(len(val_patient)))

  temp_list = pd.DataFrame({'train_roc_auc' : train_roc_auc,
                                  'train_loss' : train_loss,
                                  'val_roc_auc' : val_roc_auc,
                                  'val_loss' : val_loss,
                                  }, 
                                  columns=['train_roc_auc','train_loss', 'val_roc_auc','val_loss' ])

  temp_list.to_csv("/content/Double_CNN_LSTM_with_Attention_with_metainfo_folderNo._{}.csv".format(time), index = False)